In [346]:
import pandas as pd
import numpy as np

In [347]:
test = pd.read_csv('tajik_np_full_clinical_copy.csv')[['ID','Wave','A1','Behavior1','Behavior5','Behavior8','Behavior12',
                                                      'Behavior13','Behavior16','Behavior19','Behavior23',
                                                      'Behavior28','Behavior29','Behavior32','Behavior33',
                                                      'Behavior36','Behavior37','Behavior42','Behavior43',
                                                      'Behavior44','Behavior50','Behavior51','Behavior55',
                                                      'Behavior57','Behavior58','Behavior61','Behavior64',
                                                      'Behavior67','Behavior69','Behavior70','Behavior71',
                                                      'Behavior72','Behavior73']]
test.head()

/var/folders/71/p0nsprlx033b24hgn0jyt5n40000gp/T/ipykernel_42274/3608791094.py:1: DtypeWarning: Columns (0,295,296,297,298,299,300,301,302,303) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('tajik_np_full_clinical_copy.csv')[['ID','Wave','A1','Behavior1','Behavior5','Behavior8','Behavior12',


,ID,Wave,A1,Behavior1,Behavior5,Behavior8,Behavior12,Behavior13,Behavior16,Behavior19,...,Behavior57,Behavior58,Behavior61,Behavior64,Behavior67,Behavior69,Behavior70,Behavior71,Behavior72,Behavior73
0,1,1,25.0,2.0,2.0,3.0,3.0,32.0,3.0,3.0,...,2.0,3.0,3.0,2.0,3.0,3.0,2.0,1.0,2.0,2.0
1,2,1,26.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,...,1.0,3.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0
2,3,1,31.0,3.0,2.0,2.0,3.0,3.0,2.0,3.0,...,2.0,2.0,2.0,1.0,2.0,3.0,1.0,3.0,2.0,2.0
3,4,1,20.0,2.0,1.0,1.0,2.0,3.0,1.0,2.0,...,1.0,2.0,1.0,2.0,2.0,3.0,2.0,3.0,2.0,2.0
4,5,1,40.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,2.0


In [348]:
test['A1'].name

'A1'

In [349]:
def score_brief(df: pd.DataFrame, id_cols: list, age_col: str) -> pd.DataFrame:
    """

    Function to score responses to the BRIEF

    Must pass a dataframe containing:
        BRIEF item responses with the respective item number in the variable name
        Age of the responder
        Column names containing unique identifiers for the responder

    Returns a dataframe organized by particpant ID, containing raw scores, T-scores, and PR for:
        Inhibit
        Shift
        Emotional Control
        Self-Monitor
        Initiate
        Working Memory
        Plan/ Organize
        Task Monitor
        Org. of Materials
        Behavioral Regulation
        Metacognition
        Global Executive

    """

    

    # check df
    for col in [*id_cols,age_col]:
        if col not in df.columns:
            raise Exception(f'{col} not in Dataframe')

    try:
        df[age_col].apply(lambda x: x if np.nan else int(x))
    except:
        raise Exception(f'Could not convert age column {age_col} to integer')
    
    for col in df.drop(columns=[*id_cols,age_col]):
        item_no = int(''.join([i for i in [*col] if i.isdigit()]))
        new_name = f'b{item_no}'
        df.rename(columns={col:new_name}, inplace=True)
        for i, resp in enumerate(df[new_name]):
            if df.loc[df.index[i],new_name] not in [1,2,3,np.nan]:
                df.loc[df.index[i],new_name] = np.nan
                #print(f'Invalid value: {df.loc[df.index[i],col]}, index: {i}, column: {col}\nValue set to NaN')

    items = {
        'inhibit':['b5','b16','b29','b36','b43','b55','b58','b73'],
        'shift':['b8','b22','b32','b44','b61','b67'],
        'emotcont':['b1','b12','b19','b28','b33','b42','b51','b57','b69','b72'],
        'selfmonitor':['b13','b23','b37','b50','b64','b70'],
        'initiate':['b6','b14','b20','b25','b45','b49','b53','b62'],
        'workingmemory':['b4','b11','b17','b26','b35','b46','b56','b68'],
        'plan':['b9','b15','b21','b34','b39','b47','b54','b63','b66','b71'],
        'taskmonitor':['b2','b18','b24','b41','b52','b75'],
        'organization':['b3','b7','b30','b31','b40','b60','b65','b74']
    }

    for key, values in items.items():
        items[key] = list(set(values).intersection(set(df.columns)))

    print(items)
    
    # maybe do some piping here
    df_scored = df.assign(inhibit_raw=df[items['inhibit']].sum(axis='columns'),
                shift_raw=df[items['shift']].sum(axis='columns'),
                emotcont_raw=df[items['emotcont']].sum(axis='columns'),
                selfmonitor_raw=df[items['selfmonitor']].sum(axis='columns'),
                initiate_raw=df[items['initiate']].sum(axis='columns'),
                workingmemory_raw=df[items['workingmemory']].sum(axis='columns'),
                plan_raw=df[items['plan']].sum(axis='columns'),
                taskmonitor_raw=df[items['taskmonitor']].sum(axis='columns'),
                organization_raw=df[items['organization']].sum(axis='columns'))


    df_scored = df_scored.assign(behavereg_raw=df_scored[['inhibit_raw','shift_raw','emotcont_raw','selfmonitor_raw']].sum(axis='columns'),
                metacog_raw=df_scored[['initiate_raw','workingmemory_raw','plan_raw','taskmonitor_raw','organization_raw']].sum(axis='columns'))

    df_scored = df_scored.assign(globalexec_raw=df_scored[['behavereg_raw','metacog_raw']].sum(axis='columns'))

    # convert raw scores to t-scores and percents... shoooouuulld be easy peasy

    return df_scored



test_output = score_brief(df=test,id_cols=['ID','Wave'],age_col='A1')
test_output

{'inhibit': ['b55', 'b29', 'b73', 'b43', 'b16', 'b36', 'b5', 'b58'], 'shift': ['b8', 'b67', 'b44', 'b61', 'b32'], 'emotcont': ['b72', 'b42', 'b19', 'b33', 'b28', 'b69', 'b57', 'b1', 'b51', 'b12'], 'selfmonitor': ['b50', 'b70', 'b23', 'b37', 'b64', 'b13'], 'initiate': [], 'workingmemory': [], 'plan': ['b71'], 'taskmonitor': [], 'organization': []}


,ID,Wave,A1,b1,b5,b8,b12,b13,b16,b19,...,emotcont_raw,selfmonitor_raw,initiate_raw,workingmemory_raw,plan_raw,taskmonitor_raw,organization_raw,behavereg_raw,metacog_raw,globalexec_raw
0,1,1,25.0,2.0,2.0,3.0,3.0,NaN,3.0,3.0,...,25.0,11.0,0.0,0.0,1.0,0.0,0.0,69.0,1.0,70.0
1,2,1,26.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,...,25.0,13.0,0.0,0.0,2.0,0.0,0.0,67.0,2.0,69.0
2,3,1,31.0,3.0,2.0,2.0,3.0,3.0,2.0,3.0,...,28.0,11.0,0.0,0.0,3.0,0.0,0.0,63.0,3.0,66.0
3,4,1,20.0,2.0,1.0,1.0,2.0,3.0,1.0,2.0,...,20.0,14.0,0.0,0.0,3.0,0.0,0.0,54.0,3.0,57.0
4,5,1,40.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,...,22.0,12.0,0.0,0.0,3.0,0.0,0.0,59.0,3.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2545,306,2b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2546,307,2b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2547,308,2b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2548,267,2b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [350]:
tables = pd.read_excel('BRIEF_tables.xls',sheet_name=None)
tables['inhibit_t']

,raw_score,age_18,age_30,age_40,age_50,age_60,age_70,age_80
0,8,36,37,38,39,40,41,42
1,9,40,41,42,43,44,45,46
2,10,43,44,45,46,47,48,49
3,11,46,48,49,50,51,52,53
4,12,50,51,52,53,54,55,56
5,13,53,54,55,56,58,59,60
6,14,57,58,59,60,61,62,63
7,15,60,61,62,63,64,66,67
8,16,63,65,66,67,68,69,70
9,17,67,68,69,70,71,73,74
